<h1>scTDA Tutorial (v0.97)</h1>

scTDA is an object oriented python library for topological data analysis of high-throughput single-cell RNA-seq data. It includes tools for the preprocessing, analysis, and exploration of single-cell RNA-seq data, based on topological representations produced by the Mapper algorithm. This tutorial illustrates some of the basic features of scTDA using the human preimplantation dataset of _Petropoulos S, et al., "Single-Cell RNA-Seq Reveals Lineage and X Chromosome Dynamics in Human Preimplantation Embryos", Cell 165: 1012 - 1026 (2016)_, as a case study. The scTDA code contains more advanced options and commands that are not described in this tutorial. For more detailed documentation, please use the ipython command `?`. 

In [1]:
import scTDA

<h2>1. Pre-processing</h2>

For convenience, scTDA includes a class for preparing, normalizing, and filtering data. It takes as input one or more files with read or UMI counts. In each file, rows correspond to genes and tab-separated columns correspond to cells, except for the first column, which contains gene IDs. Files can be all from a same time point or from multiple time points. In this tutorial we consider 1,529 cells from 88 human preimplantation embryos sampled at 5 different timepoints. The dataset consists of 89 files corresponding to embryonic days 3 (13 files), 4 (16 files), 5 (24 files), 6 (18 files) and 7 (18 files). The file <a href="files/data.txt">`data.txt`</a> contains a summary of the file names, timepoints, and number of cells in each file.

In [2]:
files = []
cells = []
libs = []
days = []
with open('data.txt', 'r') as f:
    for line in f:
        sp = line[:-1].split('\t')
        files.append(sp[0])
        cells.append(int(sp[1]))
        libs.append(sp[2])
        days.append(int(sp[3]))

The class `scTDA.Preprocess` allows to read, normalize, filter, and organize the data, shaping it in the appropriate form for downstream analyses. An instance is initialized by providing a list of files containing the read or UMI counts, the corresponding timepoints and library ID's, and the number of cells in each file. If spike-in's were used in the experiment, they can be included in the analysis by specifying their common identifier with the option `spike=`,

In [3]:
p = scTDA.Preprocess(files, days, libs, cells, spike='ERCC-')

The method `scTDA.Preprocess.show_statistics()` can be used to show some simple statistics of the data:

In [4]:
p.show_statistics()

Minimum number of transcripts per cell: 17989
Minimum cell complexity: 5162


<img src="files/figs/fig1.png" style="width:500px">
<img src="files/figs/fig2.png" style="width:500px">
<img src="files/figs/fig3.png" style="width:500px">
<img src="files/figs/fig4.png" style="width:500px">
<img src="files/figs/fig5.png" style="width:500px">

From the first plot we observe that some cells have a low ratio between spike-in reads and average spike-in reads in the library, presumably due to low sequencing depth. In addition, some cells have low ratio of spike-in reads and uniquely mapped reads. This plot is useful for filtering cells out based on spike-in reads. The second plot shows the distribution of transcripts per gene. The third plot shows the fraction of cells with undetected expression of each gene (blue) and spike-in's (yellow) as a function of their average number of counts across all cells. This plot is useful for modelling the technical noise and identifying genes that have a high biological variability and low probability of drop-out. The fourth plot displays the distribution of the total number of transcripts per cell. We observe a large dispersion in this distribution, which we will reduce by subsampling counts. Finally, the last plot displays the distribution of the cell complexity across the dataset, defined as the number of genes expressed in each cell.

We can use these plots to decide the filetring and normalization strategy. In this particular example, we  subsample reads using the method `scTDA.Preprocess.subsample()`, so that all cells have the same total number of reads:

In [5]:
p.subsample(17989)

1529 cells remain after subsmapling


Here we have subsampled to the minimum number of reads per cell in the dataset. If we had subsampled to a larger number, cells that have a lower number of reads would have been removed from downstream analysis.

We can select for cells based on various criteria using the method `scTDA.Preprocess.select_cells()`,

In [6]:
p.select_cells(filterYlow=0.001, filterXlow=0.01, filterXhigh=3.0)

1415 cells were selected
(1415 cells after subsampling)


<img src="files/figs/fig6.png" style="width:500px">

The parameters `filterXlow` and `filterXhigh` set respectively lower and upper bounds in the ratio between spike-in reads and the average number of spike-in reads in the library. The parameters `filterYlow` and `filterYhigh` set respectively lower and upper bounds in the ratio between spike-in reads and uniquely mapped reads. Out of the 1,529 cells, 1,415 pass all the above filters. The cells that pass all the constraints are shown in red in the plot and are stored in the boolean array `scTDA.Preprocess.which_samples`.

We can display again the statistics, overlying in red the corresponding plots for the set of cells that pass all the constraints:

In [7]:
p.show_statistics()

Minimum number of transcripts per cell: 31244
Minimum cell complexity: 5162
Minimum number of transcripts per cell (subsampled): 17989
Minimum cell complexity (subsampled): 3429


<img src="files/figs/fig7.png" style="width:500px">
<img src="files/figs/fig8.png" style="width:500px">
<img src="files/figs/fig9.png" style="width:500px">
<img src="files/figs/fig10.png" style="width:500px">
<img src="files/figs/fig11.png" style="width:500px">

We can model the dependence of the dropout rate on the average gene expression by fitting a sigmoid function. Then assign a z-score to each gene by fitting the residuals with a normal distribution, where the standard deviation is estimated from the lower 16th percentile of the data. This procedure is implemented the method `scTDA.Preprocess.fit_sigmoid()`,

In [8]:
p.fit_sigmoid()

<img src="files/figs/fig12.png" style="width:500px">
<img src="files/figs/fig13.png" style="width:500px">

When subsampling has been applied (as in this particular example), `scTDA.Preprocess.fit_sigmoid()` makes use of the subsampled data to fit the sigmoid. Alternatively, the sigmoid can be fitted to the spike-in's by using the option `to_spikes=True`.

We can use the above sigmoidal fit to identify genes that have a low probability of drop-out and that have a higher variability than expected from purely technical noise. To that end, we make use of the method `scTDA.Preprocess.select_genes()`,

In [9]:
p.select_genes(avg_counts=2.0, min_z=3.0)

197 genes were selected


<img src="files/figs/fig14.png" style="width:500px">

In this particular example we have selected genes that have an average of 2 or more counts across the cells, and a minimum z-score of +3.0 with respect to the sigmoidal fit. The genes that pass the constrints are represented in red, and are stored in the variable `scTDA.Preprocess.which_genes`. Their names can be recovered thorugh the following command,

In [10]:
print p.genes[p.which_genes]

['ABCG2' 'ACAT2' 'ACTL8' 'ADM' 'AHNAK' 'ALPL' 'ALPP' 'ANP32E' 'ANPEP'
 'ANXA6' 'APOA1' 'AQP3' 'ARGFX' 'ARL4D' 'ASAH1' 'ATG9A' 'ATP1B1' 'ATP6V0A4'
 'B3GNT2' 'BAG4' 'BIK' 'BRDT' 'CCKBR' 'CCNA1' 'CD24' 'CD53' 'CD81' 'CD9'
 'CGA' 'CLDN10' 'CLDN4' 'CLIC1' 'CRIP1' 'CUL2' 'CYP2S1' 'CYP51A1' 'DDIT4'
 'DHCR24' 'DHCR7' 'DLG5' 'DLX3' 'DNMT1' 'DPPA5' 'DSC2' 'DUXA' 'EFNA1'
 'EGLN3' 'EMP2' 'EPCAM' 'ERRFI1' 'FABP3' 'FAM151A' 'FASN' 'FBXL20' 'FBXO5'
 'FDFT1' 'FOLR1' 'FOXR1' 'FRAT2' 'FSCN1' 'FZD5' 'GABARAPL1' 'GATA2' 'GATA3'
 'GATA6' 'GCM1' 'GDF9' 'GINS3' 'GLIPR2' 'GPATCH3' 'GPRC5A' 'GPX2' 'GRN'
 'H1F0' 'HERPUD1' 'HINT1' 'HIPK3' 'HJURP' 'HMGCS1' 'HMOX1' 'IFI30' 'IFITM1'
 'IFITM3' 'ITM2C' 'JUP' 'KLF17' 'KLF4' 'KPNA7' 'KRT18' 'KRT19' 'KRT8'
 'LCP1' 'LGMN' 'LMBR1L' 'LPCAT3' 'LRP2' 'LYN' 'MAGEA4' 'MFN1' 'MPP1'
 'MSMO1' 'MTMR14' 'MTRNR2L1' 'MVD' 'MX1' 'NFKBIA' 'NID1' 'NLRP4' 'OOEP'
 'OVOL1' 'PDCL3' 'PDGFA' 'PDXK' 'PEBP1' 'PGF' 'PHOSPHO1' 'PIM1' 'PLEC'
 'PLTP' 'PNP' 'PPP1R14A' 'PPP1R15A' 'PRAP1' 'PRSS23' 'PR

Once we are satisfied with the selection of cells, genes, and subsampling, we can use the command `scTDA.Preprocess.save()` to generate the neccessary files for downstream analyses based on our selection,

In [13]:
p.save('Embryo')

This command generates three tab-separated files, named <a href="files/Embryo.all.tsv">`Embryo.all.tsv`</a>, <a href="files/Embryo.no_subsampling.tsv">`Embryo.no_subsampling.tsv`</a>, and <a href="files/Embryo.mapper.tsv">`Embryo.mapper.tsv`</a>, where the rows correspond to selected cells. The first column of <a href="files/Embryo.all.tsv">`Embryo.all.tsv`</a> and <a href="files/Embryo.no_subsampling.tsv">`Embryo.no_subsampling.tsv`</a> contains a unique identifier of the cell, the second column contains the sampling timepoint, the third column contains the library ID, and the remaining columns contain $\textrm{log}_2(1+\textrm{TPM})$ expression values for all the genes. <a href="files/Embryo.all.tsv">`Embryo.all.tsv`</a> contains subsampled data, whereas <a href="files/Embryo.no_subsampling.tsv">`Embryo.no_subsampling.tsv`</a> constains non-subsampled data. <a href="files/Embryo.mapper.tsv">`Embryo.mapper.tsv`</a> contains $\textrm{log}_2(1+\textrm{TPM})$ expression values for the genes that were selected using the `scTDA.Preprocess.select_genes()`.

<h2>2. Topological representation</h2>

The file <a href="files/Embryo.mapper.tsv">`Embryo.mapper.tsv`</a> can be used to produce a topological representation of the dataset. The class `scTDA.TopologicalRepresentation` provides an interface to the python module <a href="https://github.com/RabadanLab/sakmapper">sakmapper</a>, which contains an implementation of the Mapper algorithm. In this example, we generate an instance of the Mapper algorithm using the first two PCA components of the data,

In [11]:
t = scTDA.TopologicalRepresentation('Embryo', lens='pca', metric='euclidean')

<img src="files/figs/fig15.png" style="width:500px">

We use the method `scTDA.TopologicalRepresentation.save()` to generate a Mapper representation of the data based in this PCA projection, using 25 x 25 bins with an average 40% overlap,

In [12]:
t.save('Embryo_pca_25_0.40', 25, 0.40);

total of 456 patches required clustering
this implies 1234 nodes in the mapper graph


The topological representation is stored in GEXF format in the file <a href="files/Embryo_pca_25_0.40.gexf">`Embryo_pca_25_0.40.gexf`</a>, and is accompanied by a JSON file <a href="files/Embryo_pca_25_0.40.json">`Embryo_pca_25_0.40.gexf.json`</a>, which specifies the cells that are associated to each node of the graph. We can visualize the topological representation using the class `scTDA.UnrootedGraph`,

In [13]:
c = scTDA.UnrootedGraph('Embryo_pca_25_0.40', 'Embryo.no_subsampling.tsv', groups=False)
c.draw('PDGFRA');

<img src="files/figs/fig16.png" style="width:500px">

where we have colored the nodes of the topological representation by the expression levels of PDGFRA.

An alternative and more convenient way of generating the topological representation is using the software <a href="http://www.ayasdi.com/">Ayasdi</a>. scTDA provides a method to parse a topological representation generated from the table <a href="files/Embryo.mapper.tsv">`Embryo.mapper.tsv`</a> using an existing Ayasdi session,

In [35]:
scTDA.ParseAyasdiGraph('Embryo_mds','1480173499692','-1429704228027513618','username','password')

This command produces the files <a href="files/Embryo_mds.gexf">`Embryo_mds.gexf`</a>, <a href="files/Embryo_mds.json">`Embryo_mds.json`</a> and <a href="files/Embryo_mds.groups.json">`Embryo_mds.groups.json`</a>. `'1480173499692'` and `'-1429704228027513618'` are the identifiers of the Ayasdi session that we are importing into scTDA. They can be obtained from the Ayasdi web interface as indicated in the following screenshot,

<img src="files/figs/fig17.png" style="width:750px">

<h2>3. Analysis</h2>

scTDA provides two classes for the analysis of single cell RNA-seq expression data: `scTDA.UnrootedGraph` and `scTDA.RootedGraph`, respectively for non-longitudinal and longitudinal single cell RNA-seq data. `scTDA.RootedGraph` is an inherited class from `scTDA.UnrootedGraph`, so all methods of `scTDA.UnrootedGraph` are also included in `scTDA.RootedGraph`, in addition to methods that are specific of `scTDA.RootedGraph`. In this tutorial we make use of `scTDA.RootedGraph`, since we are dealing with longitudinal data. The class is initialized with the files produced in the previous steps, 

In [2]:
c = scTDA.RootedGraph('Embryo_mds', 'Embryo.no_subsampling.tsv', posgl=True)

When using topological representations generated through sakmapper, the option `groups=False` should be specified when initializing the classes `scTDA.RootedGraph` or `scTDA.UnRootedGraph`. The argument `posgl=True` forces scTDA to use a pre-comuputed layout for visualization of the topological representation. To compute the visualization layout of a topological representation for the first time initialize the class using the argument `posgl=False`.

The class `scTDA.RootedGraph` includes several methods for the analysis and visualization of data. We can display some general statistics using the method `scTDA.RootedGraph.show_statistics()`,

In [3]:
c.show_statistics()

<img src="files/figs/fig18.png" style="width:500px">
<img src="files/figs/fig19.png" style="width:500px">
<img src="files/figs/fig20.png" style="width:500px">
<img src="files/figs/fig21.png" style="width:500px">

The first plot displays the distribution of the number of cells per node in the topological representation. The second plot presents the distribution of the number of common cells between nodes that share an edge in the topological representation. The third plot shows the distribution of the number of nodes that contain the same cell. The last plot shows the distribution of transcripts in $\textrm{log}_2(1+\textrm{TPM})$.

The method `SCTDA.RootedGraph.draw()` can be used to plot the topological representation colored by the expression of a given gene:

In [9]:
c.draw('DPPA5');

<img src="files/figs/fig22.png" style="width:500px">

Node sizes are proportional to the number of cells in the node. The method `SCTDA.RootedGraph.draw()` also allows to map a gene or list of genes to different color channels:

In [11]:
c.draw(['HTR3E', 'CDX1']);

<img src="files/figs/fig23.png" style="width:500px">

`SCTDA.RootedGraph.draw()` can map expression up to four different color channels (red, green, blue, and orange). It can also map lists of genes to a channel:

In [12]:
c.draw([['NTSR2', 'PCDHGA2', 'HTR3E'], 'CDX1']);

<img src="files/figs/fig24.png" style="width:500px">

`scTDA` accepts some special keywords as gene names. The keyword `'_CDR'` represents cell complexity (that is, the number of genes whose expression is detected in a cell):

In [13]:
c.draw('_CDR');

<img src="files/figs/fig25.png" style="width:500px">

Note that the topological representation in this example was built using subsampled data, so that the cell complexity did not affect the structure of the topological representation. The complexity computed by `_CDR` uses non-subsampled data in this example.

The correlation between sampling time point and cell complexity can be computed using the method `scTDA.RootedGraph.plot_CDR_correlation()`,

In [14]:
c.plot_CDR_correlation()

(3.0013078195545253e-05,
 0.0012797183719986786,
 0.25415051415110884,
 1.4214292231269905e-11,
 4.3670878039107468e-06)

<img src="files/figs/fig26.png" style="width:500px">

`scTDA.RootedGraph.plot_CDR_correlation()` returns the two parameters of the linear fit, Pearson's $r$, the $p$-value and the standard error.

The keyword `'timepoint'` can be used to color acccording to the sampling timepoint,

In [15]:
c.draw('timepoint');

<img src="files/figs/fig27.png" style="width:500px">

A remarkable feature of the topological representation, based just on expression data, is that it reproduces correctly the differentiation time course, as observed in the figure.

Similarly, the keyword `'timepoint_'` can be used to plot the density of cells that belong to a given time point,

In [16]:
c.draw('timepoint_4');

<img src="files/figs/fig28.png" style="width:500px">

scTDA determines a root node in the topological representation by maximizing the correlation between sampling time and the graph distance function. The root node corresponds to the less differentiated state. We can color the topological representation according to the distance to the root node using the keyword `'_dist_root'`,

In [17]:
c.draw('_dist_root');

<img src="files/figs/fig29.png" style="width:500px">

Correlation between sampling time point and distance to root node can be computed using the method `scTDA.RootedGraph.plot_rootlane_correlation()`,

In [18]:
c.plot_rootlane_correlation()

(6.0094790721342557,
 -15.225312506918854,
 0.90710250512996549,
 2.963126282391012e-259,
 0.10662067547117232)

<img src="files/figs/fig30.png" style="width:500px">

`scTDA.RootedGraph.plot_rootlane_correlation()` returns the two parameters of the linear fit, Pearson's $r$, the $p$-value, and the standard error.

The keyword `'lib_'` can be used to localize a given library within the topological representation,

In [19]:
c.draw('lib_E6_17');

<img src="files/figs/fig31.png" style="width:500px">

This can be useful to check for batch effects. Similarly, the keyword `'ID_'` can be used to identify a given cell within the topological representation,

In [20]:
c.draw('ID_D7_E7_19_7');

<img src="files/figs/fig32.png" style="width:500px">

The method `scTDA.RootedGraph.draw_expr_timeline()` can be used to compute the expression of a gene or list of genes as a function of the pseudo-time inferred from the distance to root function,

In [26]:
c.draw_expr_timeline('DNMT3B');

<img src="files/figs/fig33.png" style="width:500px">

We can compute the skeleton of the differentiation tree using the method `scTDA.RootedGraph.draw_skeleton()`. This method represents the skeleton colored according to a gene or list of genes, where each node in the skeleton corresponds to a set of connected nodes at the same distance of the root node in the condensed topological representation. Node sizes are proportional to the number of cells in the node, whereas edge sizes are proportional to the number of edges in the condensed topological representation connecting each pair of group of nodes,

In [22]:
c.draw_skeleton('timepoint', markpath=True);

<img src="files/figs/fig34.png" style="width:500px">

`scTDA.RootedGraph.draw_skeleton()` correctly reproduces in this example the segregation of the inner cell mass (red path) from the trophectoderm (gray path). The expression of a gene or gene list as a function of the pseudo-time across the inner cell mass path can be obtained using the command,

In [27]:
c.draw_expr_timeline('DNMT3B', path=True);

There are several quantities associated to each gene that allow to identify genes that are specific of particular cell sub-populations or time points. The first quantity that we may consider is the gene connectivity in the topological representation. This is defined as,

\begin{equation}
S(g) = \frac{N-1}{N}\sum_{i,j} p_i(g) w_{ij} p_j(g)
\end{equation}

where the sum runs over all nodes in the topological representation, $w$ is the adjacency matrix of the topological representation, $N$ the total number of nodes and $p_i(g)$ the average expression of gene $g$ on node $i$, normalized as a probablity, namely

\begin{equation}
\sum_i p_i = 1
\end{equation}

Gene connectivity allows to identify genes whose expression appears to be highly localized in the topological representation. The connectivity of a gene can be computed using the method `scTDA.RootedGraph.connectivity()`,

In [28]:
c.connectivity('PDGFRA')

0.026781135405469891

The statistical significance of a particular gene connectivity score can be assessed by means of a permutation test, where cell ID's are randomly permuted. This is implemented in the method `scTDA.RootedGraph.connectivity_pvalue()`,

In [29]:
c.connectivity_pvalue('PDGFRA', n=1000)

0.0

where `n` specifies the number of permutations. The idea is that genes with a statistically significant connectivity, like the above example, are biologically relevant at a particular stage or for a particular cell sub-population,

In [30]:
c.draw('PDGFRA');

<img src="files/figs/fig35.png" style="width:500px">

To the contrary, genes with non-significant gene connectivity in the topological representation are unspecific of any stage or cell sub-population,

In [32]:
c.connectivity_pvalue('ACTA2', n=1000)

0.23499999999999999

In [33]:
c.draw('ACTA2');

<img src="files/figs/fig36.png" style="width:500px">

Two other interesting quantities are the centroid of a gene with respect to the root node and its dispersion. These are respectively defined as

\begin{equation}
C(g)=\sum_i d_i p_i(g)
\end{equation}
\begin{equation}
D(g) = \sqrt{\sum_i (d_i - C(g))^2p_i(g)}
\end{equation}

where $d_i$ is the distance of node $i$ to the root node in the topological representation.

The centroid and dispersion of a gene or list of genes can be computed using the method `scTDA.RootedGraph.centroid()`,

In [34]:
c.centroid('PDGFRA')

[5.9490631857275655, 3.5017362498302624]

For convenience, `scTDA.RootedGraph.centroid()` expresses the centroid and dispersion in units of pseudo-time.

There are few other useful methods implemented in scTDA. `scTDA.RootedGraph.expr()` returns the number of cells on which expression of a gene or list of genes is detected,

In [35]:
c.expr('PDGFRA')

710

`scTDA.RootedGraph.delta()` returns the mean, minimum and maximum expression values of a gene or list of genes,

In [36]:
c.delta('PDGFRA')

(10.39388945620666, 0.0, 17.591164372547066)

These methods can be restricted to specific groups of nodes. Groups of nodes are specified in the file `name.groups.json` and stored in the python dictionary `scTDA.RootedGraph.dicgroups`. For instance, in this example we have defined two groups of nodes in the file `Embryo_mds.groups.json`. These can be accessed through the command

In [37]:
c.dicgroups.keys()

[u'group_2', u'group_1']

We can restrict the methods `scTDA.RootedGraph.expr()` and `scTDA.RootedGraph.delta()` to any of these groups by using the argument `group=`. For instance,

In [38]:
c.expr('PDGFRA', group='group_1')

46

In [39]:
c.delta('PDGFRA', group='group_1')

(7.1193535309912681, 0.0, 11.602032359743909)

The method `scTDA.RootedGraph.save()` creates a file `name.genes.tsv` that contains a tab separated table with all the above quantities for all the genes in the dataset, including Bejamini-Holchberg adjusted _p_-values for the gene connectivity. This method allows to restrict to genes that are expressed in more than `filtercells` cells, and whose maximum expression value is above `filterexp` in $\textrm{log}_2(1+\textrm{TPM})$ units. In addition, it allows to annotate genes according to one or more lists of genes.

In this example we have downloaded from <a href="http://www.ebi.ac.uk/QuickGO/">EMBL-EBI QuickGO</a> two tables listing the members of the gene ontologies _RNA splicing_ and _Poly-(A) RNA binding_. We can parse the genes in those tables into a dictionary that will be used by the command `scTDA.RootedGraph.save()` to annotate the genes:

In [40]:
splic = []
rna = []

f = open('GO0008380_RNA_spliciing.tsv', 'r')
for nb, line in enumerate(f):
    if nb > 0:
        sp = line.split('\t')
        splic.append(sp[2])
f.close()
splic = list(set(splic))

f = open('GO0044822_polyA_RNA_binding.tsv', 'r')
for nb, line in enumerate(f):
    if nb > 0:
        sp = line.split('\t')
        rna.append(sp[2])
f.close()
rna = list(set(rna))

annotations = {'Splicing': splic, 'RNA_binding': rna}

We use the method `scTDA.RootedGraph.save()` to create the file <a href="files/Embryo_mds.genes.tsv">`Embryo_mds.genes.tsv`</a>, where we only consider genes that are expressed in more than 4 cells and the statistical significance of the gene connectivity for each gene is assessed using 1,000 permutations,

In [5]:
c.save(n=1000, filtercells=4, filterexp=0.0, annotation=annotations);

<img src="files/figs/fig37.png" style="width:500px">
<img src="files/figs/fig38.png" style="width:500px">

`scTDA.RootedGraph.save()` produces two plots. The first plot displays the gene connectivity against the number of cells with non-zero expression for each gene. Genes with statistically significant gene connectivity (after Benjamini-Hochberg adjustement of the false discovery rate) are displayed in red. The second plot displays the number of cells with non-zero expression against the centroid and dispersion, for genes with statistically significant connectivity.

In [42]:
!head Embryo_mds.genes.tsv

Gene	Cells	Mean	Min	Max	Connectivity	p_value	q-value (BH)	Centroid	Dispersion	RNA_binding	Splicing
A1BG	541	7.49129060417	0.0	14.8875628512	0.0248937543432	0.904	0.95226798686	5.78584474587	3.75416390667	N	N
A1BG-AS1	183	3.48519450258	0.0	8.89706772809	0.0303189014814	0.251	0.400456692362	5.79902501246	3.83765613938	N	N
A1CF	141	2.57489600338	0.0	11.9710275418	0.0338832335836	0.121	0.233122366083	5.51176190725	3.9669054262	N	N
A2M	129	3.68243339829	0.0	12.4372462893	0.0301627828687	0.963	0.986440250157	6.11827609478	3.52709888378	N	N
A2M-AS1	54	1.99636223801	0.0	9.16848437163	0.0463249000877	0.225	0.369407586356	6.01642327503	3.60521441403	N	N
A2ML1	435	7.29057282155	0.0	12.7969439405	0.0269589980532	0.012	0.0383444210526	6.13931906515	3.56548502627	N	N
A2MP1	7	0.303144175054	0.0	6.34511216862	0.168933062088	0.426	0.581808158368	6.6126796184	2.76125254742	N	N
A3GALT2	113	1.35424919249	0.0	9.84270734957	0.0737582253025	0.0	0.0	4.56391375798	3.75466295309	N	N
A4GALT	1399	14.2953925217	6.

`scTDA.compare_results()` can be used to perform comparisons between the results of two analysis based on the centroid and dispersion of genes with significant gene conectivity. This is useful when evaluating the consistency of the results across different parameter choices, technical, or biological replicates, or between different experiments. In this example, we compare the results of two scTDA analyses of the same data differing in the metric that was used to build the topological representation. The representation in the analysis `Embryo_mds` was built using correlation distance, whereas the representation in the analysis `Embryo_mds_euclid` was built using Euclidean distance:

In [43]:
scTDA.compare_results('Embryo_mds', 'Embryo_mds_euclid');

Overlap between significant genes (Fisher's exact test p-value): 0.0
Pearson's correlation between centroids: (0.99142978032797191, 0.0)
Pearson's correlation between dispersions: (0.98043329096775433, 0.0)


<img src="files/figs/fig39.png" style="width:500px">
<img src="files/figs/fig40.png" style="width:500px">
<img src="files/figs/fig41.png" style="width:500px">

We can define transient subpopulations in an unsupervised way by clustering significantly localized low-dispersion genes according to their centroid in the topological representation. The optimal number of clusters can be determined using Davies-Bouldin criterion. This procedure is implemented in the method `scTDA.RootedGraph.cellular_subpopulations()`,

In [50]:
g = c.cellular_subpopulations(3.2)

<img src="files/figs/fig42.png" style="width:500px">
<img src="files/figs/fig43.png" style="width:500px">

In this example we have considered significant genes with a dispersion smaller than 3.2. `scTDA.RootedGraph.cellular_subpopulations()` produces two plots. The first plots shows Davies-Bouldin index as a funcion of the number of clusters. The second plot displays the distribution of low-dispersion genes according to their centroid and dispersion, and colors the different clusters. `scTDA.RootedGraph.cellular_subpopulations()` returns the lists of genes in each cluster,

In [52]:
c.draw([g[0], g[1], g[2]]);

<img src="files/figs/fig44.png" style="width:500px">

An alternative approach to identify transient cellular sub-populations is to use colocalization patterns. These can be computed by clustering the Jensen-Shannon distance matrix associated to significantly-localized low-dispersion genes,

\begin{equation}
J(g,h) = \left[\frac12\sum_i\left(p_i(g)\textrm{log }p_i(g)+p_i(h)\textrm{log }p_i(h)-(p_i(g)+p_i(h))\textrm{log }\frac{p_i(g)+p_i(h)}{2}\right)\right]^{\frac12}
\end{equation}

To that end, we use the command:

In [11]:
g = c.cellular_subpopulations(3.15, method='js', clus_thres=0.7)

<img src="files/figs/fig45.png" style="width:500px">

In [12]:
c.draw([g[0],g[1],g[2]]);

<img src="files/figs/fig46.png" style="width:500px">